
Based on ddpg_training.py

This script is used to visualize the inference from a DDPG model trained via PPO for pose prediction using purely visual inputs. 


## Skip this cell unless you have physical access to the robot

In [ ]:
# Skip this part unless you have physical access to the robot

import os
import time
import rclpy
from random_motion_logger import RandomMotionLogger
from dead_reckoning import FinalPoseProcessor
import torch

# Import PPO inference function here
from infer_utils import (
    infer_poses,
    load_actor,
    load_feature_extractor,
    NUM_FRAMES,
    HISTORY_LENGTH,
)


NUM_FRAMES = 32
IMAGE_DIR = '/home/earl/workspaces/rldr/infer_trajectory'

def collect_random_trajectory(node):
    """
    Moves the robot randomly and collects 32 images.
    Returns a list of image file paths.
    """
    node.start_new_trajectory()
    while node.frame_count < NUM_FRAMES:
        rclpy.spin_once(node, timeout_sec=0.1)
    
    # Collect the image paths
    image_paths = [os.path.join(node.traj_path, f"frame_{i:04d}.jpg") for i in range(NUM_FRAMES)]
    return 



rclpy.init()
# 1. Start random motion and collect images
random_motion_node = RandomMotionLogger()


Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
xFormers not available
xFormers not available
INFO:dinov2:using MLP layer as FFN


Exception in thread [INFO] [1748853535.169975595] [random_motion_logger]: Waiting for ENTER key to start a new trajectory...
Thread-5 (keypress_listener):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/earl/workspaces/turtlevision/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/earl/workspaces/rldr/random_motion_logger.py", line 118, in keypress_listener
    old_settings = termios.tcgetattr(fd)
termios.error: (25, 'Inappropriate ioctl for device')


nvbufsurftransform: Could not get EGL display connection


## Start here without access to the robot

In [1]:
# Inference batch over all trajectories

import os
import torch
import pandas as pd
from infer_utils import infer_poses, load_actor, load_feature_extractor, NUM_FRAMES, HISTORY_LENGTH

# Configuration
TRAJ_ROOT      = '/home/earl/workspaces/rldr/trajectories'
#CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/best_ddpg_2.pt'
#CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/rosy-rain-39-8.pt'
#CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/rosy-rain-39-41.pt'
#CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/rosy-rain-39-64.pt'
CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/rosy-rain-39-86.pt'
#CHECKPOINT_PATH= '/home/earl/workspaces/rldr/infer_trajectory/rosy-rain-39-173.pt'

# 1. Load model & feature extractor once
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
actor  = load_actor(CHECKPOINT_PATH, device, history_length=HISTORY_LENGTH)
feat_ex= load_feature_extractor(device)

# 2. Iterate over each traj folder, run inference, collect final poses
records = []
for traj in sorted(os.listdir(TRAJ_ROOT)):
    traj_path = os.path.join(TRAJ_ROOT, traj)
    if not os.path.isdir(traj_path):
        continue

    # collect up to NUM_FRAMES images
    image_paths = [
        os.path.join(traj_path, fname)
        for fname in sorted(os.listdir(traj_path))
        if fname.endswith('.jpg')
    ][:NUM_FRAMES]
    if len(image_paths) < NUM_FRAMES:
        print(f"Skipping {traj}: only {len(image_paths)} images")
        continue

    # run inference
    poses = infer_poses(image_paths, actor, feat_ex, device, history_length=HISTORY_LENGTH)
    pred_x, pred_y, pred_theta = poses[-1]

    records.append({
        "traj_id": traj,
        "x_pred":  pred_x,
        "y_pred":  pred_y,
        "theta_pred": pred_theta
    })
    print(f"{traj}: x={pred_x:.3f}, y={pred_y:.3f}, θ={pred_theta:.1f}°")

# 3. Build DataFrame of all final‐pose predictions
df_preds = pd.DataFrame(records)
df_preds

/home/earl/workspaces/rldr/infer_utils.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  actor.load_state_dict(torch.load(checkpoint_path, map_location=device))


traj_000: x=1.000, y=-0.448, θ=-180.0°
traj_001: x=1.000, y=0.984, θ=-176.3°
traj_002: x=0.999, y=-0.999, θ=-180.0°
traj_003: x=1.000, y=-0.988, θ=-180.0°
traj_004: x=1.000, y=0.963, θ=-176.9°
traj_005: x=1.000, y=0.999, θ=-141.8°
traj_006: x=1.000, y=-0.848, θ=-179.9°
traj_007: x=1.000, y=0.950, θ=-179.5°
traj_008: x=1.000, y=0.999, θ=32.0°
traj_009: x=1.000, y=0.801, θ=-180.0°
traj_010: x=1.000, y=0.926, θ=-178.6°
traj_011: x=1.000, y=0.928, θ=-179.4°
traj_012: x=1.000, y=0.997, θ=-156.3°
traj_013: x=1.000, y=-0.961, θ=-180.0°
traj_014: x=1.000, y=1.000, θ=53.0°
traj_015: x=1.000, y=-0.996, θ=-180.0°
traj_016: x=0.999, y=-0.999, θ=-180.0°
traj_017: x=1.000, y=0.069, θ=-180.0°
traj_018: x=1.000, y=0.989, θ=-176.8°
traj_019: x=1.000, y=-0.995, θ=-180.0°
traj_020: x=1.000, y=-0.350, θ=-180.0°
traj_021: x=1.000, y=0.516, θ=-179.9°
traj_022: x=1.000, y=0.980, θ=-175.4°
traj_023: x=1.000, y=0.175, θ=-179.9°
traj_024: x=0.999, y=-0.999, θ=-180.0°
traj_025: x=1.000, y=0.990, θ=-179.5°
traj_0

,traj_id,x_pred,y_pred,theta_pred
0,traj_000,0.999947,-0.447614,-179.981300
1,traj_001,0.999974,0.984058,-176.270303
2,traj_002,0.999472,-0.999071,-179.988745
3,traj_003,0.999894,-0.988125,-179.995151
4,traj_004,0.999975,0.963363,-176.882726
...,...,...,...,...
1277,traj_995,0.999949,0.793749,-179.812320
1278,traj_996,0.999667,0.999621,74.388835
1279,traj_997,0.999955,-0.790349,-179.995097
1280,traj_998,0.999951,-0.303505,-179.986085


## Visualize the distribution of predicted poses over a discretized action space

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

# === Config ===
GRID_SIZE = 5
CELL_SIZE = 0.3 # m
NUM_ORIENT = 72
LAMBDA_POS = 1.0
LAMBDA_ORI = 2.0

def action_to_pose(action_id):
    ang = action_id % NUM_ORIENT
    idx = action_id // NUM_ORIENT
    row, col = divmod(idx, GRID_SIZE)
    x = (col - GRID_SIZE // 2) * CELL_SIZE
    y = (row - GRID_SIZE // 2) * CELL_SIZE
    theta = ang * (360.0 / NUM_ORIENT)
    return [x, y, theta]

def pose_error(pred, gt, λ_pos=LAMBDA_POS, λ_ori=LAMBDA_ORI):
    dx, dy = pred[0] - gt[0], pred[1] - gt[1]
    pos_err = (dx**2 + dy**2)**0.5
    ori_err = abs((pred[2] - gt[2] + 180) % 360 - 180)
    return λ_pos * pos_err**2 + λ_ori * (ori_err / 180.0)**2

def analyze_logs(root_dir):
    records = []
    # grid covers [−(GRID_SIZE*CELL_SIZE)/2, +(GRID_SIZE*CELL_SIZE)/2]
    offset = (GRID_SIZE * CELL_SIZE) / 2.0

    for traj_folder in sorted(os.listdir(root_dir)):
        traj_path = os.path.join(root_dir, traj_folder)
        log_path  = os.path.join(traj_path, "log.csv")
        if not os.path.isfile(log_path):
            continue

        try:
            df = pd.read_csv(log_path)
            for _, row in df.iterrows():
                if 'action_id' not in row or pd.isna(row['action_id']):
                    continue

                action_id = int(row["action_id"])
                gt_x  = float(row["rel_x"])
                gt_y  = float(row["rel_y"])
                gt_th = float(row["rel_theta"])

                # compute pose error
                pred_pose = action_to_pose(action_id)
                err = pose_error(pred_pose, [gt_x, gt_y, gt_th])

                # convert ground‐truth (x,y) into 5×5 cell bins
                x_bin = int(np.floor((gt_x + offset) / CELL_SIZE))
                y_bin = int(np.floor((gt_y + offset) / CELL_SIZE))
                x_bin = min(max(x_bin, 0), GRID_SIZE-1)
                y_bin = min(max(y_bin, 0), GRID_SIZE-1)

                theta_bin = action_id % NUM_ORIENT

                records.append({
                    "x_bin":      x_bin,
                    "y_bin":      y_bin,
                    "theta_bin":  theta_bin,
                    "pose_error": err
                })
        except Exception as e:
            print(f"[WARN] Skipped {traj_path}: {e}")

    return pd.DataFrame(records)

def plot_pose_error_maps(df):
    # Heatmap of average pose error per (x, y)
    plt.figure(figsize=(6, 5))
    heatmap = df.groupby(["y_bin", "x_bin"])["pose_error"].mean().unstack()
    sns.heatmap(heatmap, annot=True, fmt=".3f", cmap="Reds")
    plt.title("Average Pose Error per (x, y) Bin")
    plt.xlabel("x_bin")
    plt.ylabel("y_bin")
    plt.tight_layout()
    plt.show()

    # Barplot of average pose error per theta_bin
    plt.figure(figsize=(10, 4))
    theta_avg = df.groupby("theta_bin")["pose_error"].mean()
    sns.barplot(x=theta_avg.index, y=theta_avg.values, color="salmon")
    plt.title("Average Pose Error per Orientation Bin")
    plt.xlabel("theta_bin")
    plt.ylabel("Avg Pose Error")
    plt.tight_layout()
    plt.show()

def plot_cell_orientation_histograms(df):
    """
    Plot a 5×5 grid of cells where:
      - x (row index) is vertical ↑ (north),
      - y (col index) is horizontal → (east).
    Each cell shows a histogram of orientation angles in [-180,180],
    and the bottom row displays x-ticks.
    Cell background color encodes sample count.
    """
    # compute counts per cell
    counts_xy = (
        df.groupby(['x_bin','y_bin'])
          .size()
          .unstack(fill_value=0)
          .reindex(index=range(GRID_SIZE),
                   columns=range(GRID_SIZE),
                   fill_value=0)
    )

    # collect raw 0→360° angles per cell
    angles_xy = {
        (x, y): (
            df.loc[(df.x_bin==x)&(df.y_bin==y), 'theta_bin']
              * (360.0/NUM_ORIENT)
        ).values
        for x in range(GRID_SIZE)
        for y in range(GRID_SIZE)
    }

    # prepare colormap
    vmax = counts_xy.values.max()
    norm = mcolors.Normalize(vmin=0, vmax=vmax)
    cmap = plt.cm.Reds

    # square figure + constrained layout for roughly square subplots
    fig, axes = plt.subplots(
        GRID_SIZE, GRID_SIZE,
        figsize=(10,10),
        subplot_kw={'projection':'polar'},
        constrained_layout=True
    )

    # build 13 bins around the circle
    num_bins  = 26
    bin_edges = np.linspace(0, 2*np.pi, num_bins+1)

    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            ax = axes[GRID_SIZE-1-x, y]
            thetas = np.deg2rad(angles_xy[(x, y)])
            ax.hist(thetas, bins=bin_edges, color='k', alpha=0.7)
            ax.set_yticks([])
            ax.set_theta_zero_location('N')   # zero at top
            ax.set_theta_direction(-1)        # clockwise
            ax.set_facecolor(cmap(norm(counts_xy.at[x,y])))

    # Overlay a grid on the array of rose plots
    for i in range(GRID_SIZE+1):
        # Horizontal lines
        fig.add_artist(plt.Line2D(
            [0, 1], [i / GRID_SIZE, i / GRID_SIZE],
            color='gray', linewidth=1, alpha=0.7, transform=fig.transFigure, zorder=100
        ))
        # Vertical lines
        fig.add_artist(plt.Line2D(
            [i / GRID_SIZE, i / GRID_SIZE], [0, 1],
            color='gray', linewidth=1, alpha=0.7, transform=fig.transFigure, zorder=100
        ))

    # Adjust layout to make space for suptitle and colorbar
    plt.subplots_adjust(top=1.90, bottom=0.10)

    # main title
    plt.suptitle('Orientation Distributions (0° at top center)',y=1.02)

    # colorbar in the newly freed space at the very bottom
    sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar_ax = fig.add_axes([0.15, -0.07, 0.7, 0.025])  # [left, bottom, width, height]
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_label_position('top')
    cbar.set_label('Sample Count')

    plt.show()

# Prepare df_preds for plot_cell_orientation_histograms

import numpy as np

# === reuse CONFIG from above cells ===
GRID_SIZE   = 5
CELL_SIZE   = 0.3     # m
NUM_ORIENT  = 72

# center‐offset to convert from [−half, +half] coords → bin index
offset   = (GRID_SIZE * CELL_SIZE) / 2.0
bin_size = 360.0 / NUM_ORIENT

# assume df_preds has columns ['traj_id','x_pred','y_pred','theta_pred']
# 1) compute x_bin, y_bin
df_preds['x_bin'] = (
    np.floor((df_preds['x_pred'] + offset) / CELL_SIZE)
    .clip(0, GRID_SIZE - 1)
    .astype(int)
)
df_preds['y_bin'] = (
    np.floor((df_preds['y_pred'] + offset) / CELL_SIZE)
    .clip(0, GRID_SIZE - 1)
    .astype(int)
)

# 2) compute theta_bin
df_preds['theta_bin'] = (
    (df_preds['theta_pred'] / bin_size)
    .round()
    .astype(int)
    % NUM_ORIENT
)

# 3) we don’t need pose_error here – plot_cell_orientation_histograms only uses x_bin,y_bin,theta_bin

# 4) call the plotting function
plot_cell_orientation_histograms(df_preds)    

## Calculate the errors in pose deltas

In [23]:
# Table of error comparing predictions with ground truth for all traj_id
# Each traj_id has a file named 'final_pose.txt' with one row that specifies pose as x,y,theta
def load_final_poses(traj_root):
    records = []
    for traj in sorted(os.listdir(traj_root)):
        traj_path = os.path.join(traj_root, traj)
        final_pose_file = os.path.join(traj_path, 'final_pose.txt')
        if not os.path.isfile(final_pose_file):
            continue
        
        with open(final_pose_file, 'r') as f:
            line = f.readline().strip()
            x, y, theta = map(float, line.split(' '))
            records.append({
                "traj_id": traj,
                "x_gt": x,
                "y_gt": y,
                "theta_gt": theta
            })
    
    return pd.DataFrame(records)

# Load ground truth final poses
gt_poses = load_final_poses(TRAJ_ROOT)
# Merge predictions with ground truth
df_results = pd.merge(df_preds, gt_poses, on='traj_id', how='left')
# Compute errors
df_results['x_error'] = df_results['x_pred'] - df_results['x_gt']
df_results['y_error'] = df_results['y_pred'] - df_results['y_gt']
df_results['theta_error'] = df_results['theta_pred'] - df_results['theta_gt']
# Print the final results DataFrame
print(df_results[['traj_id', 'x_pred', 'y_pred', 'theta_pred', 'x_gt', 'y_gt', 'theta_gt', 'x_error', 'y_error', 'theta_error']])

# Make a summary table of errors
def summarize_errors(df):
    summary = {
        "mean_x_error": df['x_error'].mean(),
        "mean_y_error": df['y_error'].mean(),
        "mean_theta_error": df['theta_error'].mean(),
        "std_x_error": df['x_error'].std(),
        "std_y_error": df['y_error'].std(),
        "std_theta_error": df['theta_error'].std(),
        "max_x_error": df['x_error'].max(),
        "max_y_error": df['y_error'].max(),
        "max_theta_error": df['theta_error'].max()
    }
    return summary
# Summarize errors
error_summary = summarize_errors(df_results)
# Print the error summary
print("Error Summary:")
for key, value in error_summary.items():
    print(f"{key}: {value:.3f}")
    

       traj_id    x_pred    y_pred  theta_pred      x_gt      y_gt  theta_gt  \
0     traj_000  0.999947 -0.447614 -179.981300  0.559051  0.159237     49.86   
1     traj_001  0.999974  0.984058 -176.270303  0.538714  0.106217    321.54   
2     traj_002  0.999472 -0.999071 -179.988745  0.372062 -0.288953    285.47   
3     traj_003  0.999894 -0.988125 -179.995151  0.453001 -0.032601    351.74   
4     traj_004  0.999975  0.963363 -176.882726  0.601438 -0.085554     34.03   
...        ...       ...       ...         ...       ...       ...       ...   
1277  traj_995  0.999949  0.793749 -179.812320  0.437090 -0.025578    314.25   
1278  traj_996  0.999667  0.999621   74.388835  0.415528  0.181956     80.67   
1279  traj_997  0.999955 -0.790349 -179.995097  0.394330  0.118126    355.28   
1280  traj_998  0.999951 -0.303505 -179.986085  0.360688  0.361974     53.38   
1281  traj_999  0.999933 -0.277790 -179.994603  0.591639 -0.009732    345.71   

       x_error   y_error  theta_error  